In [1]:
TPU = 'grpc://10.209.42.186:8470'
BATCH_SIZE = 32
IMAGE_COUNT = 1790594
IMAGE_SIZE = 256
IMAGE_TRAIN_LEN = 56

LOSS_WEIGHTS = {
    'pe_present_on_image' : 0.0736196319 * 10, 
    'negative_exam_for_pe' : 0.0736196319,
    'indeterminate' : 0.09202453988,
    'chronic_pe' : 0.1042944785,
    'acute_and_chronic_pe' : 0.1042944785,
    'central_pe' : 0.1877300613,
    'leftsided_pe' : 0.06257668712 ,
    'rightsided_pe' : 0.06257668712,
    'rv_lv_ratio_gte_1' : 0.2346625767,
    'rv_lv_ratio_lt_1' : 0.0782208589,
}

In [2]:
import os
os.sys.path.append('../input/rsna-sped-ds-utility/')
import glob
import math
import numpy as np 
import pandas as pd 
import tensorflow as tf
import matplotlib.pyplot as plt
import efficientnet.tfkeras as efn 

from tqdm import tqdm
from tensorflow.keras import layers, backend, models
# from kaggle_datasets import KaggleDatasets
from sklearn.model_selection import KFold

ModuleNotFoundError: No module named 'efficientnet'

In [3]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [4]:
# try:
# #     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu=TPU,
#                                                            zone='europe-west4-a',
#                                                             project='dfdc-264702')
#     print('Running on TPU ', tpu.master())
# except ValueError:
#     tpu = None

# if tpu:
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.experimental.TPUStrategy(tpu)
# else:
#     strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

# print("REPLICAS: ", strategy.num_replicas_in_sync)

In [5]:
train_df = pd.read_csv('../input/rsna-str-pulmonary-embolism-detection/train.csv')

In [6]:
cols = train_df.columns.values
study_cols = list(cols[4:])

In [7]:
# GCS_DS_PATH_TRAIN = KaggleDatasets().get_gcs_path('rsna-sped-ds-1011')
GCS_DS_PATH_TRAIN = 'gs://rsna-sped/data_256_w'

In [8]:
# fs = glob.glob('../input/rsna-sped-ds-1011/*.tfrec')
fs = glob.glob('../data_256_w/*.tfrec')
fs = sorted(fs)
fs = [GCS_DS_PATH_TRAIN + '/' + os.path.basename(f) for f in fs]
fs = np.array(fs)

In [9]:
AUTO = tf.data.experimental.AUTOTUNE

# kf = KFold(n_splits=5, random_state=23, shuffle=True)
kf = KFold(n_splits=5)
for train_index, valid_index in kf.split(fs):
    break
valid_index

NameError: name 'KFold' is not defined

In [10]:
def read_tfrecord(data):
    features = {
        "study_labels": tf.io.FixedLenFeature([13], tf.int64), 
        "image_labels": tf.io.FixedLenSequenceFeature([], tf.int64, allow_missing=True), 
        "images": tf.io.FixedLenFeature([], tf.string), 
    }

    # decode the TFRecord
    tf_record = tf.io.parse_single_example(data, features)
    
    study_labels = tf_record['study_labels']
    study_labels = tf.cast(study_labels, tf.float32)
    image_labels = tf_record['image_labels']
    image_labels = tf.cast(image_labels, tf.float32)
    
    images_bytes = tf.io.parse_tensor(tf_record['images'], tf.string)
    
    start = tf.constant(0, tf.int64)
    end = tf.cast(tf.shape(image_labels)[0], tf.int64)
    
    t = tf.where(image_labels>0)
    def gt():
        return backend.min(t), backend.max(t)
    
    def le():
        s = tf.random.uniform(shape=[], minval=backend.max(t) - IMAGE_TRAIN_LEN, maxval=backend.min(t), dtype=tf.int64)
        s = tf.cond(s < 0, lambda: tf.constant(0, tf.int64), lambda: s)
        s = tf.cond(s + IMAGE_TRAIN_LEN < end, lambda: s, lambda: end - IMAGE_TRAIN_LEN)
        e = s + IMAGE_TRAIN_LEN
        return s, e
    
    def pe():
        return tf.cond(backend.max(t) - backend.min(t) > IMAGE_TRAIN_LEN, gt, le)
    
    def no_pe():
        return start, end
    
    start, end = tf.cond(tf.greater(backend.sum(image_labels), tf.constant(0, tf.float32)), pe, no_pe)
    
    image_labels = image_labels[start:end]
    images_bytes = images_bytes[start:end]
    
#     if np.sum(t[0]) > 0:
#         tt = tf.where(t[0]>0)
#         print(len(t[0]),  backend.max(tt) - backend.min(tt))


    l = tf.cast(tf.shape(image_labels)[0], tf.int64)
    n = l // IMAGE_TRAIN_LEN
    
    s = tf.cond(tf.equal(l%IMAGE_TRAIN_LEN, tf.constant(0, tf.int64)), 
                lambda: tf.constant(0, tf.int64), 
                lambda: tf.constant(1, tf.int64))
    
#     s = tf.random.uniform([], maxval=l-IMAGE_TRAIN_LEN*n+1, dtype=tf.int64)
    i = tf.range(s, l, delta=n, dtype=tf.int64)
    image_labels = tf.gather(image_labels, i[:IMAGE_TRAIN_LEN])
    images_bytes = tf.gather(images_bytes, i[:IMAGE_TRAIN_LEN])
    
    images = []
    for b in tf.unstack(images_bytes, num=IMAGE_TRAIN_LEN):
        image = tf.image.decode_jpeg(b)
        image = tf.image.random_jpeg_quality(image, 30, 100)
        images.append(image)
    del images_bytes
    images = tf.stack(images)
    images = tf.cast(images, tf.float32) / 256
    images = tf.reshape(images, (IMAGE_TRAIN_LEN, IMAGE_SIZE, IMAGE_SIZE, 3))
    
    return images, (image_labels, 
                    study_labels[study_cols.index('negative_exam_for_pe')],
                    study_labels[study_cols.index('indeterminate')], 
                    study_labels[study_cols.index('chronic_pe')], 
                    study_labels[study_cols.index('acute_and_chronic_pe')], 
                    study_labels[study_cols.index('central_pe')], 
                    study_labels[study_cols.index('leftsided_pe')], 
                    study_labels[study_cols.index('rightsided_pe')], 
                    study_labels[study_cols.index('rv_lv_ratio_gte_1')], 
                    study_labels[study_cols.index('rv_lv_ratio_lt_1')])

In [11]:
# dataset = tf.data.TFRecordDataset(fs)
# dataset = dataset.map(read_tfrecord)

In [12]:
# for index, (images, (t)) in enumerate(dataset.as_numpy_iterator()):
# #     if t[1] == 0 and np.sum(t[0]) == 0:
# #         print(t[0])
#     if np.sum(t[0]) > 0:
#         tt = tf.where(t[0]>0)
#         print(len(t[0]),  backend.max(tt) - backend.min(tt))
#         break

In [13]:
# https://www.kaggle.com/cdeotte/rotation-augmentation-gpu-tpu-0-96
def get_mat(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    # returns 3x3 transformmatrix which transforms indicies
        
    # CONVERT DEGREES TO RADIANS
    rotation = math.pi * rotation / 180.
    shear = math.pi * shear / 180.
    
    # ROTATION MATRIX
    c1 = tf.math.cos(rotation)
    s1 = tf.math.sin(rotation)
    one = tf.constant([1],dtype='float32')
    zero = tf.constant([0],dtype='float32')
    rotation_matrix = tf.reshape( tf.concat([c1,s1,zero, -s1,c1,zero, zero,zero,one],axis=0),[3,3] )
        
    # SHEAR MATRIX
    c2 = tf.math.cos(shear)
    s2 = tf.math.sin(shear)
    shear_matrix = tf.reshape( tf.concat([one,s2,zero, zero,c2,zero, zero,zero,one],axis=0),[3,3] )    
    
    # ZOOM MATRIX
    zoom_matrix = tf.reshape( tf.concat([one/height_zoom,zero,zero, zero,one/width_zoom,zero, zero,zero,one],axis=0),[3,3] )
    
    # SHIFT MATRIX
    shift_matrix = tf.reshape( tf.concat([one,zero,height_shift, zero,one,width_shift, zero,zero,one],axis=0),[3,3] )
    
    return backend.dot(backend.dot(rotation_matrix, shear_matrix), backend.dot(zoom_matrix, shift_matrix))

def transform(images, l):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SIZE
    XDIM = DIM%2 #fix for size 331
    
    rot = 10. * tf.random.normal([1],dtype='float32')
    shr = 3. * tf.random.normal([1],dtype='float32') 
    h_zoom = 1.0 + tf.random.normal([1],dtype='float32') / 20.
    w_zoom = 1.0 + tf.random.normal([1],dtype='float32') / 20.
    h_shift = 8. * tf.random.normal([1],dtype='float32') 
    w_shift = 8. * tf.random.normal([1],dtype='float32') 
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = backend.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = backend.cast(idx2,dtype='int32')
    idx2 = backend.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    idx3 = tf.transpose(idx3)
    
    images = tf.transpose(images, perm=[1, 2, 0, 3])
    images = tf.gather_nd(images, idx3)
    images = tf.transpose(images, perm=[1, 0, 2])
    images = tf.reshape(images, [IMAGE_TRAIN_LEN, DIM, DIM, 3])
              
    return images, l

In [14]:
# images = tf.constant(np.zeros((56, 256, 256, 3)))

# idx3 = tf.constant(np.zeros((65536, 2)), dtype=tf.int64)
# tf.gather_nd(images, idx3)

In [15]:
# dataset = tf.data.TFRecordDataset(fs)
# dataset = dataset.map(read_tfrecord)
# dataset = dataset.apply(tf.data.experimental.ignore_errors())
# for i, (images, labels) in enumerate(dataset):
#     print(i)
# #     break

In [16]:
# labels

In [17]:
# for i in images:
#     plt.imshow(i[:,:,0])
#     plt.show()
# #     break

In [18]:
train_dataset = tf.data.TFRecordDataset(fs[train_index])
train_dataset = train_dataset.map(read_tfrecord)
train_dataset = train_dataset.map(transform)
train_dataset = train_dataset.apply(tf.data.experimental.ignore_errors())
train_dataset = train_dataset.prefetch(AUTO)
train_dataset = train_dataset.repeat()
train_dataset = train_dataset.batch(BATCH_SIZE)

valid_dataset = tf.data.TFRecordDataset(fs[valid_index])
valid_dataset = valid_dataset.map(read_tfrecord)
valid_dataset = valid_dataset.apply(tf.data.experimental.ignore_errors())
valid_dataset = valid_dataset.prefetch(AUTO)
valid_dataset = valid_dataset.batch(BATCH_SIZE)

NameError: name 'train_index' is not defined

In [19]:
for index, (images, (t)) in enumerate(train_dataset.as_numpy_iterator()):
#     if t[1] == 0 and np.sum(t[0]) == 0:
#         print(t[0])
    break

NameError: name 'train_dataset' is not defined

In [20]:
for image in images[0]:
    plt.imshow(image)
    plt.show()

NameError: name 'images' is not defined

In [21]:
def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters
    if backend.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = layers.Conv3D(filters1, (1, 1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv3D(filters2, kernel_size,
                      padding='same',
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv3D(filters3, (1, 1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    x = layers.add([x, input_tensor])
    x = layers.Activation('relu')(x)
    return x


def conv_block(input_tensor,
               kernel_size,
               filters,
               stage,
               block,
               strides=(1, 2, 2)):
    """A block that has a conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: default 3, the kernel size of
            middle conv layer at main path
        filters: list of integers, the filters of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
        strides: Strides for the first conv layer in the block.
    # Returns
        Output tensor for the block.
    Note that from stage 3,
    the first conv layer at main path is with strides=(2, 2)
    And the shortcut should have strides=(2, 2) as well
    """
    filters1, filters2, filters3 = filters
    if backend.image_data_format() == 'channels_last':
        bn_axis = 3
    else:
        bn_axis = 1
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'

    x = layers.Conv3D(filters1, (1, 1, 1), strides=strides,
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2a')(input_tensor)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2a')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv3D(filters2, kernel_size, padding='same',
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2b')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2b')(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv3D(filters3, (1, 1, 1),
                      kernel_initializer='he_normal',
                      name=conv_name_base + '2c')(x)
    x = layers.BatchNormalization(axis=bn_axis, name=bn_name_base + '2c')(x)

    shortcut = layers.Conv3D(filters3, (1, 1, 1), strides=strides,
                             kernel_initializer='he_normal',
                             name=conv_name_base + '1')(input_tensor)
    shortcut = layers.BatchNormalization(
        axis=bn_axis, name=bn_name_base + '1')(shortcut)

    x = layers.add([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

In [22]:
with strategy.scope():
    def create_model():
        n = 32
        x_input = layers.Input(shape=(None, 256, 256, 3))
        x = x_input
        x = layers.ZeroPadding3D(padding=(0, 3, 3), name='conv1_pad')(x)
        x = layers.Conv3D(64, (1, 7, 7),
                              strides=(1, 2, 2),
                              padding='valid',
                              kernel_initializer='he_normal',
                              name='conv1')(x)
        x = layers.BatchNormalization(name='bn_conv1')(x)
        x = layers.Activation('relu')(x)
        x = layers.ZeroPadding3D(padding=(0, 1, 1), name='pool1_pad')(x)
        x = layers.MaxPooling3D((1, 3, 3), strides=(1, 2, 2))(x)
        
        x = conv_block(x, (1, 3, 3), [n, n, n*4], stage=2, block='a', strides=(1, 1, 1))
        x = identity_block(x, (1, 3, 3), [n, n, n*4], stage=2, block='b')
        x = identity_block(x, (1, 3, 3), [n, n, n*4], stage=2, block='c')

        x = conv_block(x, (1, 3, 3), [n*2, n*2, n*8], stage=3, block='a')
        x = identity_block(x, (1, 3, 3), [n*2, n*2, n*8], stage=3, block='b')
        x = identity_block(x, (1, 3, 3), [n*2, n*2, n*8], stage=3, block='c')
        x = identity_block(x, (1, 3, 3), [n*2, n*2, n*8], stage=3, block='d')

        x = conv_block(x, 3, [n*4, n*4, n*16], stage=4, block='a')
        x = identity_block(x, 3, [n*4, n*4, n*16], stage=4, block='b')
        x = identity_block(x, 3, [n*4, n*4, n*16], stage=4, block='c')
        x = identity_block(x, 3, [n*4, n*4, n*16], stage=4, block='d')
        x = identity_block(x, 3, [n*4, n*4, n*16], stage=4, block='e')
        x = identity_block(x, 3, [n*4, n*4, n*16], stage=4, block='f')

        x = conv_block(x, 3, [n*8, n*8, n*32], stage=5, block='a')
        x = identity_block(x, 3, [n*8, n*8, n*32], stage=5, block='b')
        x = identity_block(x, 3, [n*8, n*8, n*32], stage=5, block='c')
        
        x_i = backend.max(x, axis=[2, 3], keepdims=True)
        x_i = tf.keras.layers.Dense(1, activation='sigmoid')(x_i)
#         pe_present_on_image = tf.keras.layers.Reshape([IMAGE_TRAIN_LEN], name='pe_present_on_image')(x_i)
        pe_present_on_image = tf.keras.layers.Lambda(lambda x: tf.squeeze(x, axis=[2,3,4]), name='pe_present_on_image')(x_i)
        
        x_s = layers.GlobalMaxPool3D()(x)
        
        negative_exam_for_pe = tf.keras.layers.Dense(1, activation='sigmoid', name='negative_exam_for_pe')(x_s)
        indeterminate = tf.keras.layers.Dense(1, activation='sigmoid', name='indeterminate')(x_s)
        chronic_pe = tf.keras.layers.Dense(1, activation='sigmoid', name='chronic_pe')(x_s)
        acute_and_chronic_pe = tf.keras.layers.Dense(1, activation='sigmoid', name='acute_and_chronic_pe')(x_s)
        central_pe = tf.keras.layers.Dense(1, activation='sigmoid', name='central_pe')(x_s)
        leftsided_pe = tf.keras.layers.Dense(1, activation='sigmoid', name='leftsided_pe')(x_s)
        rightsided_pe = tf.keras.layers.Dense(1, activation='sigmoid', name='rightsided_pe')(x_s)
        rv_lv_ratio_gte_1 = tf.keras.layers.Dense(1, activation='sigmoid', name='rv_lv_ratio_gte_1')(x_s)
        rv_lv_ratio_lt_1 = tf.keras.layers.Dense(1, activation='sigmoid', name='rv_lv_ratio_lt_1')(x_s)
        
        model = tf.keras.models.Model(x_input, [pe_present_on_image, 
                                                negative_exam_for_pe,
                                                indeterminate,
                                                chronic_pe,
                                                acute_and_chronic_pe,
                                                central_pe,
                                                leftsided_pe,
                                                rightsided_pe,
                                                rv_lv_ratio_gte_1,
                                                rv_lv_ratio_lt_1
                                               ])
#         optimizer = tf.keras.optimizers.Adam(0.0005)
        optimizer = tf.keras.optimizers.Adamax()
        loss = tf.keras.losses.BinaryCrossentropy()
        model.compile(optimizer, loss, loss_weights=LOSS_WEIGHTS)
        return model 

    model = create_model()
    model.summary(line_length=130)

NameError: name 'layers' is not defined

In [23]:
class ValidCB(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        model.save('resnet3d_{:03d}_{:.05f}.h5'.format(epoch, logs['val_loss']))
        print('')
        
model.fit(train_dataset,
          validation_data=valid_dataset,
          steps_per_epoch=int(7279 * 0.8 / BATCH_SIZE),
          epochs=100,
          verbose=1,
          callbacks=[ValidCB()],
         )

NameError: name 'model' is not defined